In [6]:
import numpy as np
import cv2
from PIL import Image
import os
path = 'db'
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("C:\\Users\Sinagam Pradeep\Anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml");
# function to get the images and label data
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L') 
        img_numpy = np.array(PIL_img,'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)
    return faceSamples,ids
faces,ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))
# Save the model into trainer/trainer.yml
recognizer.write('trainer/trainer.yml') 
# Print the numer of faces trained and end program
print(" {0} face trained".format(len(np.unique(ids))))

 2 face trained


In [7]:
!pip install imageai

In [8]:
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
faceCascade = cv2.CascadeClassifier("C:\\Users\Sinagam Pradeep\Anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml");
font = cv2.FONT_HERSHEY_SIMPLEX
from imageai.Detection import VideoObjectDetection
import os
path = os.getcwd()
# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
ids=[333,150]
name=["pradeep","no liveness detected"]
while True:
    ret, img =cam.read()
   
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
       )
    if(len(faces)==0):
        cv2.putText(
                    img, 
                    "NO Person", 
            (40,20),font, 
                    1, 
                    (255,255,255), 
                    2)

    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        if(confidence<60 or confidence>75):
        
                cv2.putText(
                    img, 
                    name[ids.index(id)], 
                    (x+5,y-5), 
                    font, 
                    1, 
                    (255,255,255), 
                    2
                   )

        else:
            cv2.putText(
                    img, 
                    "Another Person Detected", 
                    (x+5,y-5), 
                    font, 
                    1, 
                    (255,255,255), 
                    2
                   )

    cv2.imshow('camera',img)
    key=cv2.waitKey(1) 
    if(key==ord("s")): 
        break
cam.release()
cv2.destroyAllWindows()